In [58]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
from numbers import Number


## Kilka rozszerzeń klasy **CategoricalNB**
**Cel:**  
- Klasa powinna obsługiwać klasyfikację danych kategorycznych - poprzez dedykowany obiekt klasy **OrdinalEncoder**

In [59]:

data = pd.read_csv("../data/credit-g.csv")
data = pd.read_csv("../data/golf_df.csv")
print(data)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
ylab = np.unique(y)[::-1]
y = (y == 'no').astype(int)
print("Decyzje: ",ylab)

     Outlook Temperature Humidity  Windy Play
0      sunny         hot     high  False   no
1      sunny         hot     high   True   no
2   overcast         hot     high  False  yes
3      rainy        mild     high  False  yes
4      rainy        cool   normal  False  yes
5      rainy        cool   normal   True   no
6   overcast        cool   normal   True  yes
7      sunny        mild     high  False   no
8      sunny        cool   normal  False  yes
9      rainy        mild   normal  False  yes
10     sunny        mild   normal   True  yes
11  overcast        mild     high   True  yes
12  overcast         hot   normal  False  yes
13     rainy        mild     high   True   no
Decyzje:  ['yes' 'no']


In [60]:
enc = OrdinalEncoder()
enc.fit(X)
print("Attributes:")
for cat in enc.categories_:
    print(cat)
X_ = enc.transform(X)
print("Data:", X_, sep="\n")

Attributes:
['overcast' 'rainy' 'sunny']
['cool' 'hot' 'mild']
['high' 'normal']
[False True]
Data:
[[2. 1. 0. 0.]
 [2. 1. 0. 1.]
 [0. 1. 0. 0.]
 [1. 2. 0. 0.]
 [1. 0. 1. 0.]
 [1. 0. 1. 1.]
 [0. 0. 1. 1.]
 [2. 2. 0. 0.]
 [2. 0. 1. 0.]
 [1. 2. 1. 0.]
 [2. 2. 1. 1.]
 [0. 2. 0. 1.]
 [0. 1. 1. 0.]
 [1. 2. 0. 1.]]


**Cel (cd):**  

- Akceptacja danych *częściowych*: brak wartości zgłaszany jako `np.nan` 
- Akceptacja danych *nierozpoznanych* - niedostępnych w trakcie **fit()** 

In [61]:
x = np.asarray(["sunny","hot","normal",True], dtype=object)
# without 'dtype' the last value is changed to string 'True' 
x_ = enc.transform(x[None,:])
print(x_)
x3 = X[3].copy()
x3[2] = np.nan
# x3_ = enc.transform(x3[None,:])  
# ValueError: Found unknown categories [nan] in column 2 during transform
# ... but - after a small change ...
enc.handle_unknown = 'use_encoded_value'
x3_ = enc.transform(x3[None,:])  
x4 = X[4:5].copy()
x4[0,0] = "fine"
x4_ = enc.transform(x4)
print(x3_,x4_)
print(X[4])

[[2. 1. 1. 1.]]
[[ 1.  2. nan  0.]] [[nan  0.  1.  0.]]
['rainy' 'cool' 'normal' False]


Wartość parametru `.handle_unknown = 'use_encoded_value'` blokuje z kolei akceptowanie wartości `np.nan` w fazie **fit()**. Stąd pomysł na rozszerzenie klasy OrdinalEncoder:

In [62]:
class OrdinalNanEncoder(OrdinalEncoder):    # Dopuszcza nan's dla uczących i dla transformacji
                                            # W trakcie uczenia ewent. None's traktuje jak etykiety
    def fit(self, X, y=None):
        self.handle_unknown = "error"
        return super().fit(X,y)

    def transform(self, X):
        self.handle_unknown = 'use_encoded_value'
        return super().transform(X)
